In [61]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset
import torchattacks
from torchsummary import summary
from torchvision import models, datasets, transforms
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import math

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


import sys
from watermarkJPEG import *
from JPEG_layer import * # some utils.

In [2]:
transform = transforms.Compose(
        [
            #transforms.Resize(256),
            #transforms.CenterCrop(224),
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

dataset = torchvision.datasets.ImageNet(root="./data/", split='val',
                             transform=transform
                                       )

test_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [4]:
class Interpolate(nn.Module):
    def __init__(self,size=(224,224), mode='bilinear') -> None:
        super(Interpolate, self).__init__()
        self.size = size
        self.mode = mode
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        #input = torch.squeeze(input)
        #return torch.unsqueeze(F.interpolate(input, size=self.size, mode=self.mode), dim=0)
        return F.interpolate(input, size=self.size, mode=self.mode)

In [5]:
def modelIntegration(pretrained_model):
    class IntegratedModel(nn.Module):
        def __init__(self, pretrained_model):
            super(IntegratedModel, self).__init__()
            interpolate = Interpolate()
            self.Integrated_Model = nn.Sequential(
                interpolate,
                pretrained_model
            )

        def forward(self, x):
            x = self.Integrated_Model(x)
            return x
    return IntegratedModel(pretrained_model)

In [6]:
pretrained_model = models.resnet18(pretrained=True,  progress = True).to(device)
#_ = pretrained_model.eval()

In [7]:
IntegratedModel = modelIntegration(pretrained_model).to(device)
_=IntegratedModel.eval()

In [8]:
def recreate_image(im_as_var):
    """
        Recreates images from a torch variable, sort of reverse preprocessing

    Args:
        im_as_var (torch variable): Image to recreate

    returns:
        recreated_im (numpy arr): Recreated image in array
    """
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    recreated_im = torch.clone(im_as_var)
    recreated_im = recreated_im.cpu().numpy()[0]
    for c in range(3):
        recreated_im[c] /= reverse_std[c]
        recreated_im[c] -= reverse_mean[c]
    recreated_im[recreated_im > 1] = 1
    recreated_im[recreated_im < 0] = 0
    #recreated_im = np.round(recreated_im * 255)
    recreated_im = recreated_im.transpose(1, 2, 0)
    #recreated_im = np.uint8(recreated_im).transpose(1, 2, 0)
    # Convert RBG to GBR
    #recreated_im = recreated_im[..., ::-1]
    return recreated_im

In [9]:
def recreate_image(im_as_var):
    recreated_im = torch.clone(im_as_var)
    recreated_im = recreated_im.cpu().numpy()[0]
    #recreated_im = np.round(recreated_im * 255)
    recreated_im = recreated_im.transpose(1, 2, 0)
    #recreated_im = np.uint8(recreated_im).transpose(1, 2, 0)
    # Convert RBG to GBR
    #recreated_im = recreated_im[..., ::-1]
    return recreated_im

In [10]:
attack = torchattacks.attacks.pgd.PGD(IntegratedModel,
                                      eps=8/255,
                                      alpha=1/255)

In [12]:
def blockify(im: torch.Tensor, size: int) -> torch.Tensor:
    r"""
    Breaks an image into non-overlapping blocks of equal size.

    Parameters
    ----------
    im : Tensor
        The image to break into blocks, must be in :math:`(N, C, H, W)` format.
    size : Tuple[int, int]
        The size of the blocks in :math:`(H, W)` format.

    Returns
    -------
    A tensor containing the non-overlappng blocks in :math:`(N, C, L, H, W)` format where :math:`L` is the
    number of non-overlapping blocks in the image channel indexed by :math:`(N, C)` and :math:`(H, W)` matches
    the block size.

    Note
    ----
    If the image does not split evenly into blocks of the given size, the result will have some overlap. It
    is the callers responsibility to pad the input to a multiple of the block size, no error will be thrown
    in this case.
    """
    bs = im.shape[0]
    ch = im.shape[1]
    h = im.shape[2]
    w = im.shape[3]

    im = im.reshape(bs * ch, 1, h, w)
    im = torch.nn.functional.unfold(im, kernel_size=(size, size), stride=(size, size))
    im = im.transpose(1, 2)
    im = im.reshape(bs, ch, -1, size, size)

    return im

In [13]:
def _normalize(N: int) -> torch.Tensor:
    r"""
    Computes the constant scale factor which makes the DCT orthonormal
    """
    n = torch.ones((N, 1))
    n[0, 0] = 1 / math.sqrt(2)
    return n @ n.t()

In [14]:
def _harmonics(N: int) -> torch.Tensor:
    r"""
    Computes the cosine harmonics for the DCT transform
    """
    spatial = torch.arange(float(N)).reshape((N, 1))
    spectral = torch.arange(float(N)).reshape((1, N))

    spatial = 2 * spatial + 1
    spectral = (spectral * math.pi) / (2 * N)

    return torch.cos(spatial @ spectral)

In [15]:
def block_dct(blocks: torch.Tensor) -> torch.Tensor:
    r"""
    Computes the DCT of image blocks

    Args:
        blocks (Tensor): Non-overlapping blocks to perform the DCT on in :math:`(N, C, L, H, W)` format.
    
    Returns:
        Tensor: The DCT coefficients of each block in the same shape as the input.

    Note:
        The function computes the forward DCT on each block given by 

        .. math::

            D_{i,j}={\frac {1}{\sqrt{2N}}}\alpha (i)\alpha (j)\sum _{x=0}^{N}\sum _{y=0}^{N}I_{x,y}\cos \left[{\frac {(2x+1)i\pi }{2N}}\right]\cos \left[{\frac {(2y+1)j\pi }{2N}}\right]
        
        Where :math:`i,j` are the spatial frequency indices, :math:`N` is the block size and :math:`I` is the image with pixel positions :math:`x, y`. 
        
        :math:`\alpha` is a scale factor which ensures the transform is orthonormal given by 

        .. math::

            \alpha(u) = \begin{cases}{
                    \frac{1}{\sqrt{2}}} &{\text{if }}u=0 \\
                    1 &{\text{otherwise}}
                \end{cases}
        
        There is technically no restriction on the range of pixel values but to match JPEG it is recommended to use the range [-128, 127].
    """
    N = blocks.shape[3]

    n = _normalize(N).float()
    h = _harmonics(N).float()

    if blocks.is_cuda:
        n = n.cuda()
        h = h.cuda()
    
    coeff = (1 / math.sqrt(2 * N)) * n * (h.t() @ blocks @ h)

    return coeff

In [46]:
def rgb_to_ycbcr(image: torch.Tensor) -> torch.Tensor:
    r"""Convert an RGB image to YCbCr.

    Args:
        image (torch.Tensor): RGB Image to be converted to YCbCr.

    Returns:
        torch.Tensor: YCbCr version of the image.
    """

    if not torch.is_tensor(image):
        raise TypeError("Input type is not a torch.Tensor. Got {}".format(
            type(image)))

    if len(image.shape) < 3 or image.shape[-3] != 3:
        raise ValueError("Input size must have a shape of (*, 3, H, W). Got {}"
                         .format(image.shape))

    r: torch.Tensor = image[..., 0, :, :]
    g: torch.Tensor = image[..., 1, :, :]
    b: torch.Tensor = image[..., 2, :, :]

    delta = .5
    y: torch.Tensor = .299 * r + .587 * g + .114 * b
    cb: torch.Tensor = (b - y) * .564 + delta
    cr: torch.Tensor = (r - y) * .713 + delta
    return torch.stack((y, cb, cr), -3)

In [76]:
correct_cnt = 0
block_cnt = 0
cum_turb = torch.zeros((8,8))
for _ in tqdm((range(len(test_loader)))):
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    #data.requires_grad = True
    output = IntegratedModel(data)
    init_pred = output.max(1, keepdim=True)[1]
    attack_img = attack(data, init_pred[0])
    # we assume that thh accuracy of the model is 1
    #print((attack_img - data).shape)
    #turb_noise = (attack_img - data).cpu().numpy()[0].transpose(1, 2, 0)
    
    turb_noise = (attack_img - data)[0]
    
    #print('turb_noise.shape',turb_noise.shape)
    turb_noise = rgb_to_ycbcr(turb_noise)
    #print('turb_noise.shape',turb_noise.shape,torch.max(turb_noise),torch.min(turb_noise))
    
    blocks = blockify(torch.unsqueeze(torch.unsqueeze(turb_noise[0],dim=0),dim=0),8) # get the Y channel, then reshape it to [N,C,H,W]
    dct_cof = block_dct(blocks)
    dct_cof = torch.squeeze(dct_cof)
    cum_turb += torch.sum(torch.abs(dct_cof),dim = 0)
    #print(dct_cof.shape,img_Y.shape)
    #print(cum_turb)
    if _ == 10:
        break
cum_turb

  0%|          | 10/50000 [00:29<40:32:51,  2.92s/it]


tensor([[1285.3757,  853.8616,  727.3779,  571.3262,  446.0864,  423.0541,
          358.6108,  266.9424],
        [ 862.3959,  730.6536,  643.5001,  508.7614,  407.0836,  380.1028,
          321.3410,  242.9688],
        [ 733.6719,  646.3253,  560.1807,  455.8930,  367.6834,  340.5405,
          290.7578,  226.7159],
        [ 594.9268,  540.8038,  466.2228,  377.1771,  310.1926,  293.3833,
          256.3384,  210.4578],
        [ 467.8964,  431.9413,  378.3402,  311.0285,  266.3231,  251.2716,
          226.5678,  197.8513],
        [ 407.3125,  375.2373,  332.2826,  280.7406,  245.2440,  232.6831,
          212.3874,  190.2050],
        [ 350.6968,  323.6541,  290.4131,  254.4960,  226.7369,  216.0959,
          199.6210,  183.3576],
        [ 301.0378,  271.9758,  251.7820,  228.7316,  209.4221,  200.5654,
          189.5780,  178.0936]])

In [77]:
%pwd

'/media/deponce/Passport/ECE699'